In [23]:
%run PSO.ipynb #da qui possiamousare la funzione PSO

In [24]:
import keras
import numpy as np
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical, plot_model
from keras.preprocessing.text import Tokenizer
from keras.optimizers import SGD, Adam, RMSprop
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,Flatten
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [25]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000, test_split=0.2)


In [26]:
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

8982 train sequences
2246 test sequences


In [27]:
num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

46 classes


In [28]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=10000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (8982, 10000)
x_test shape: (2246, 10000)


In [29]:
print('Convert class vector to binary class matrix for use with categorical_crossentropy')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix for use with categorical_crossentropy
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [30]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

def MLP(particle_position):
    '''This function returns the accuracy of a neural network model with the parameter set as the component of the particle position.
     The first component of the position will be the batch size, the second willl be the optimizer and the third will be the learning rate. 
     particle_position[0]: batch_size
     particle_position[1]: optimizer (<1: SGD; within [1,2): SGD with Momentum; within [2,3): Adam; within [3,inf]: RMSPROP )
     particle_position[2]: learning_rate'''
    
    batch_s = particle_position[0]
    batch_s = int(batch_s)
    optimiz = particle_position[1]
    lr = particle_position[2]
    
    model = Sequential()
    model.add(Dense(512, input_shape=(10000,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    early_stopping = EarlyStopping(monitor='val_loss',patience=3, verbose=1, mode='min', baseline=None)
    callbacks = [early_stopping]

    if optimiz < 1:
        #use SGD
        opt = SGD(learning_rate = lr)
        
    elif optimiz >=1 and optimiz < 2:
        #use SGD with Momentum
        opt = SGD(momentum = 0.9, learning_rate = lr)

    elif optimiz >= 2 and optimiz < 3:
        #use ADAM
        opt = Adam(learning_rate = lr)

    else:
        #use RMSPROP
        opt = RMSprop(learning_rate = lr)

    model.compile(optimizer= opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=batch_s,
                    validation_split=0.2,
                    callbacks=callbacks)
    
    last_accuracy = history.history['val_accuracy'][-1]

    return last_accuracy

In [31]:
'''plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()'''

"plt.plot(history.history['loss'], label='Training loss')\nplt.plot(history.history['val_loss'], label='Validation loss')\nplt.title('Training and validation loss')\nplt.xlabel('Epochs')\nplt.ylabel('Loss')\nplt.legend()\nplt.show()"

In [32]:
'''plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()'''

"plt.plot(history.history['accuracy'], label='Training accuracy')\nplt.plot(history.history['val_accuracy'], label='Validation accuracy')\nplt.title('Training and validation accuracy')\nplt.xlabel('Epochs')\nplt.ylabel('Accuracy')\nplt.legend()\nplt.show()"

In [33]:
evaluation, best_parameter, best_accuracy = PSO(swarm_size = 10,
                                    dim = 3,
                                    evaluation_funct = MLP,
                                    lower_bound=[0,0,0.0001],
                                    upper_bound = [128,4,0.01],
                                    v_max = [64,2,0.001],
                                    problem = 'maximum',
                                    max_iteration = 10,
                                    termination_criteria = ['fixed_iteration', 'swarm_radius'],
                                    max_tol = 0.1)

print('Best position found: ', best_parameter, 'with an accuracy of: ',best_accuracy)

Particle position:  [array([8.28217056e+01, 2.18845348e+00, 4.99891222e-04]), array([1.11513740e+02, 3.72618625e+00, 4.93325568e-03]), array([1.48091005e+01, 1.48195919e+00, 9.98973628e-03]), array([2.08924972e+01, 1.21491175e+00, 7.69937363e-03]), array([9.62912151e+01, 3.26904331e+00, 2.09942819e-03]), array([2.34021735e+01, 9.70347612e-01, 6.11996194e-03]), array([6.85107263e+01, 1.71137038e+00, 9.07482981e-04]), array([1.53387845e+01, 3.75111905e+00, 1.80295165e-03]), array([4.80770726e+01, 2.58978011e+00, 9.30276952e-03]), array([5.75690838e+01, 4.77660140e-01, 9.15625997e-03])]
Epoch 1/20
88/88 [==============================] - 10s 89ms/step - loss: 1.8698 - accuracy: 0.6093 - val_loss: 1.1763 - val_accuracy: 0.7496
Epoch 2/20
88/88 [==============================] - 7s 74ms/step - loss: 0.8753 - accuracy: 0.8104 - val_loss: 0.9429 - val_accuracy: 0.8002
Epoch 3/20
88/88 [==============================] - 5s 52ms/step - loss: 0.5479 - accuracy: 0.8875 - val_loss: 0.8454 - val_ac

In [ ]:
# Let's plot the best evaluation function (in this case the best accuracy) found for each iteration.
plt.plot(list(range(len(evaluation))), evaluation)
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.show()